In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('covid_toy.csv')

In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [6]:
X_train

,age,gender,fever,cough,city
88,5,Female,100.0,Mild,Kolkata
98,5,Female,98.0,Strong,Mumbai
93,27,Male,100.0,Mild,Kolkata
26,19,Female,100.0,Mild,Kolkata
30,15,Male,101.0,Mild,Delhi
...,...,...,...,...,...
71,75,Female,104.0,Strong,Delhi
85,16,Female,103.0,Mild,Bangalore
7,20,Female,NaN,Strong,Mumbai
74,34,Female,104.0,Strong,Delhi


Without Column Transformer -- So hectic work

In [7]:
# adding simple imputer to fever col as it has some missing values
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
                                 
X_train_fever.shape

(80, 1)

In [8]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [10]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [11]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [13]:
# This is the time when we have to merge all numpy arrays to big numpy arrays
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed

array([[  5.        , 100.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   0.        ],
       [  5.        ,  98.        ,   0.        ,   0.        ,
          0.        ,   1.        ,   1.        ],
       [ 27.        , 100.        ,   1.        ,   0.        ,
          1.        ,   0.        ,   0.        ],
       [ 19.        , 100.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   0.        ],
       [ 15.        , 101.        ,   1.        ,   1.        ,
          0.        ,   0.        ,   0.        ],
       [ 20.        , 102.        ,   1.        ,   1.        ,
          0.        ,   0.        ,   1.        ],
       [ 18.        , 104.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 14.        , 101.        ,   1.        ,   0.        ,
          0.        ,   0.        ,   1.        ],
       [ 38.        , 101.        ,   0.        ,   0.        ,
          0.    

With Column Transformer -- No hectic work

In [15]:
from sklearn.compose import ColumnTransformer

In [17]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

# passthrough means for remaining column don't do anything -- like in our case "age"

In [22]:
transformer.fit_transform(X_train)

array([[100.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,   5.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          0.        ,   1.        ,   5.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  27.        ],
       [100.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  19.        ],
       [101.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  15.        ],
       [102.        ,   1.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  20.        ],
       [104.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  18.        ],
       [101.        ,   1.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  14.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          0.    

In [25]:
transformer.transform(X_test)

array([[102.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   1.        ,  74.        ],
       [101.        ,   1.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  68.        ],
       [104.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,  12.        ],
       [102.        ,   0.        ,   0.        ,   1.        ,
          0.        ,   0.        ,  49.        ],
       [ 98.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  26.        ],
       [103.        ,   1.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  70.        ],
       [ 98.        ,   1.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  10.        ],
       [103.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  48.        ],
       [100.76388889,   1.        ,   1.        ,   0.        ,
          1.    